In [1]:
import pandas as pd
import snowflake.connector
from collections import defaultdict
import json
import re

con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')

cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [2]:
sql = '''
with cf as (
select
  DOMAIN,
  STATUS,
  CATALOG_ID,
  CLAIMED_AT,
  COMPLETED_AT,
  AUDIT_LEVEL,
  CREATED_AT,
  CUSTOMER_FEEDBACK_DATE_SENT,
  RESULT as json_values
from
  PUBLIC.SPOTTERAUDITS
WHERE
--  STATUS = 'Accepted'
    AUDIT_TYPE = 'CustomerFeedback' AND
    AUDIT_LEVEL = 'Attempter' AND
    STATUS = 'Submitted To Review' AND
    CUSTOMER_FEEDBACK_DATE_SENT > '2022-09-16'
)

select
--    cf.DOMAIN,
    cf.STATUS,
--    cf.CATALOG_ID,
    cf.AUDIT_LEVEL,
    cf.CREATED_AT,
--    cf.CLAIMED_AT,
--    cf.COMPLETED_AT,
    cf.CUSTOMER_FEEDBACK_DATE_SENT,
    a.key PRODUCT_ID,
    b.key ATTRIBUTE,
    c.value AUDIT_RESULT
--  c.key AUDIT,
--  c.value RES
FROM
    cf,
    lateral flatten (input => json_values) a,
    lateral flatten (input => a.value, outer => true) b,
    lateral flatten (input => b.value, outer => true) c
--    lateral flatten (input => c.value, outer
WHERE AUDIT_RESULT !='' AND AUDIT_RESULT !='true' 
AND AUDIT_RESULT !='false' 
-- AND CUSTOMER_FEEDBACK_DATE_SENT > '2022-09-12' 
AND AUDIT_RESULT LIKE '%Non-Reproducible%' 
OR AUDIT_RESULT LIKE '%Reproducible%' 
OR AUDIT_RESULT LIKE '%Fully Correct by the Customer%' 
OR AUDIT_RESULT LIKE '%Audit%'
'''
cs.execute(sql)

df = cs.fetch_pandas_all()

In [3]:
df.groupby('CUSTOMER_FEEDBACK_DATE_SENT')['PRODUCT_ID'].nunique()

CUSTOMER_FEEDBACK_DATE_SENT
2022-09-18     784
2022-09-19     335
2022-09-20    8514
2022-09-21    1619
2022-09-22    2158
2022-09-23    1621
2022-09-24      24
2022-09-25    2218
2022-09-26     384
Name: PRODUCT_ID, dtype: int64

In [4]:
print(len(df))
#df.to_csv('audits0824.csv')

127619


In [ ]:
list(df.columns)

In [6]:
df.groupby('ATTRIBUTE').count()

,STATUS,AUDIT_LEVEL,CREATED_AT,CUSTOMER_FEEDBACK_DATE_SENT,PRODUCT_ID,AUDIT_RESULT
ATTRIBUTE,,,,,,
description,17721,17721,17721,17721,17721,17721
image_link,17723,17723,17723,17723,17723,17723
title,17723,17723,17723,17723,17723,17723


In [ ]:
#df = df.drop('CREATED_AT', axis=1)

In [ ]:
#df.loc[df['PRODUCT_ID'] == '4200195373417638']

In [5]:
#df.drop(df[df['STATUS'] != 'Submitted To Review'].index, inplace = True)
#df.drop(df[df['CUSTOMER_FEEDBACK_DATE_SENT'] == '2022-08-15'].index, inplace = True)
df.drop(df[df['ATTRIBUTE'] == 'brand'].index, inplace = True)
df.drop(df[df['ATTRIBUTE'] == 'additional_image_link'].index, inplace = True)
df.drop(df[df['ATTRIBUTE'] == 'color'].index, inplace = True)
df.drop(df[df['ATTRIBUTE'] == 'dimensions'].index, inplace = True)
df.drop(df[df['ATTRIBUTE'] == 'gemstone'].index, inplace = True)
df.drop(df[df['ATTRIBUTE'] == 'material'].index, inplace = True)
df.drop(df[df['ATTRIBUTE'] == 'pattern'].index, inplace = True)
df.drop(df[df['ATTRIBUTE'] == 'product_height'].index, inplace = True)
df.drop(df[df['ATTRIBUTE'] == 'product_length'].index, inplace = True)
df.drop(df[df['ATTRIBUTE'] == 'product_width'].index, inplace = True)
df.drop(df[df['ATTRIBUTE'] == 'scent'].index, inplace = True)
df.drop(df[df['ATTRIBUTE'] == 'size'].index, inplace = True)

In [7]:
df.PRODUCT_ID.nunique()

16064

In [8]:
print(len(df))

53167


In [ ]:
#df.to_csv('audits0925.csv')

In [9]:
sql = '''
select
CATALOG_ID, PRODUCT_ID from PUBLIC.CUSTOMERINPUTS
'''
cs.execute(sql)

cid = cs.fetch_pandas_all()

In [10]:
print(len(cid))

50110290


In [11]:
mergedf = df.merge(cid, left_on='PRODUCT_ID', right_on='PRODUCT_ID', how='inner', indicator=True)

In [12]:
print(len(mergedf))

53659


In [13]:
mergedf.to_csv('mergecid0927.csv')

In [ ]:
sql = '''
select
  CAST(catalog_id as varchar) as CATALOG_ID,
  CAST(product_id as varchar) as PRODUCT_ID,
  ATTRIBUTE_LABELS,
  DECISION_DATE
FROM
  PC_FIVETRAN_DB.GSHEETS.flamingocfapril_25
WHERE
  CUSTOMER_DATE_SENT > '2022-09-11'
''' 
cs.execute(sql)
rawdf = cs.fetch_pandas_all()

In [ ]:
for i in range(len(rawdf)):
    df_x = json.loads(rawdf['ATTRIBUTE_LABELS'][i])
rawdf = rawdf.update(df_x)